In [1]:
from ortools.sat.python import cp_model
import pandas as pd
import plotly.express as px

In [2]:
### Variables ###

In [3]:
days = range(5) ###Eventually model will be extended to a month or year
sessions = range(17)
tracers = range(4)

###TODO: Add variable for second PET scanner and staff schedules; Design matrix?
#Human Readable
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
sess_times = ["08:00 - 10:00","08:30 - 10:30","09:00 - 11:00",
             "09:30 - 11:30","10:00 - 12:00","10:30 - 12:30",
             "11:00 - 13:00","11:30 - 13:30","12:00 - 14:00",
             "12:30 - 14:30","13:00 - 15:00","13:30 - 15:30",
             "14:00 - 16:00","14:30 - 16:30","15:00 - 17:00",
             "15:30 - 17:30","16:00 - 18:00"]
tracers_names = ["PIB","AV1451","MK6240","UCB-J"]

In [4]:
### No use for design matrix yet but could be used to 
#weight individual sessions
first = [1,1,1,1,1,
         1,1,1,1,1,
         1,1,1,1,1,
         1,1]

second = [first, first, first, first, first]
third = [second, second, second, second]
#print(third)

In [5]:
### Creates the model CP Model: 
#https://developers.google.com/optimization/reference/python/sat/python/cp_model
model = cp_model.CpModel()

In [6]:
### Decision Variables ###

In [7]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is a Bool of whether day/session/tracer is
#instance is scheduled or not
x = {}
for t in tracers:
    for d in days:
        for s in sessions:
            x[(t,d,s)] = model.NewBoolVar(
                "{}{}{}".format(tracers_names[t],
                                day_names[d],
                                sess_times[s]))

In [8]:
### Hard Constraints ###

In [9]:
#Constraints creating distinct sessions

In [10]:
#Require each day to have at least n scans
#Push solver to schedule scans since no objective function

In [11]:
#Each session gets at most 1 tracer
for d in days:
    for s in sessions:
        model.Add(sum(x[(t,d,s)] for t in tracers) <= 1)

In [12]:
#2 hr slots without overlap
for d in days:
    for s in sessions[0:14]:
        overlap = []
        for t in tracers:
            overlap.extend([
                x[(t,d,s)],x[(t,d,s+1)],
                x[(t,d,s+2)],x[(t,d,s+3)]
            ])
        model.Add(sum(overlap) <= 1)

In [13]:
#Each tracer gets at least 1 session per week
for t in tracers:
    all_sess = []
    for d in days:
        for s in sessions:
            all_sess.append(x[(t,d,s)])
    model.Add(sum(all_sess) >= 1)

In [14]:
#PIB constraints

In [15]:
#PIB 3 max per day/batch
t = tracers_names.index("PIB")
for d in days[1:4]:
    p_day = []
    for s in sessions:
        p_day.append(x[(t,d,s)])
        model.Add(sum(p_day) <= 3)

In [16]:
#PIB 1 max per Monday
t = tracers_names.index("PIB")
d = day_names.index("Mon")
p_mon = []
for s in sessions:
    p_day.append(x[(t,d,s)])
model.Add(sum(p_day) <= 1)

In [17]:
#PIB 3 hr gap between PIB scans
t = tracers_names.index("PIB")
for d in days:
    for s in sessions[0:12]:
        p_gap = []
        for num in range(6):
            p_gap.append(x[(t,d,s+num)])
        model.Add(sum(p_gap) <= 1)

In [18]:
#AV1451 constraints

In [19]:
#12 per month for AV1451; Avgerage to 3 per week for weekly model
t = tracers_names.index("AV1451")
a_week = []
for d in days:
    for s in sessions:
        a_week.append(x[(t,d,s)])
model.Add(sum(a_week) <= 3)

In [20]:
#24HR between production but 2 scans per production batch/day 
t = tracers_names.index("AV1451")

for d in days:
    a_2 = []
    for s in sessions:
        a_2.append(x[(t,d,s)])
    model.Add(sum(a_2) <= 2)

In [21]:
#12:00 earliest sess for AV1451
t = tracers_names.index("AV1451")
a_early = []
for d in days:
    for s in sessions[0:8]:
        a_early.append(x[(t,d,s)])
        #print("x{}{}{}".format(d,s,t)) #sum() = 0
model.Add(sum(a_early) == 0)

In [22]:
#MK6240 constraints

In [23]:
#MK6240 max 3 sessions each day
t = tracers_names.index("MK6240")
for d in days:
    m_3 = []
    for s in sessions:
        m_3.append(x[(t,d,s)])
    model.Add(sum(m_3) <= 3)

In [24]:
#MK6240 only sessions 10 through 16; Tues-Fri
t = tracers_names.index("MK6240")
m_days = []
#sess_not = [0,1,2,3,4,5,6,7,8,16]
for d in days[1:5]:
    for s in list(sessions[0:9])+list(sessions[16:18]):
        m_days.append(x[(t,d,s)])
model.Add(sum(m_days) == 0)

In [25]:
#MK6240 only sessions 8 through 13; Mon
t = tracers_names.index("MK6240")
d = day_names.index("Mon")
m_mon = []
for s in list(sessions[0:7])+list(sessions[13:18]):
    m_mon.append(x[(t,d,s)])
model.Add(sum(m_mon) == 0)

In [26]:
#UCB-J constraints

In [27]:
#UCB-J max 1 session each day
t = tracers_names.index("UCB-J")
for d in days:
    u_1 = []
    for s in sessions:
        u_1.append(x[(t,d,s)])
    model.Add(sum(u_1) <= 1)

In [28]:
#UCB-J session 6 or 12 Tues-Fri
t = tracers_names.index("UCB-J")
for d in days[1:5]:
    u_sess = []
    for s in list(sessions[0:5])+list(sessions[6:11])+list(sessions[12:18]):
        u_sess.append(x[(t,d,s)])
    model.Add(sum(u_sess) == 0)

In [29]:
#UCB-J session 14 Mon
t = tracers_names.index("UCB-J")
d = day_names.index("Mon")
u_mon = []
for s in list(sessions[0:13])+list(sessions[14:18]):
    u_mon.append(x[(t,d,s)])
model.Add(sum(u_mon) == 0)

In [30]:
### Instanciate Solver, solve, and output end states ###

In [31]:
#Class used from Google OR Docs Employee Scheduling Examples
###TODO: Rename for tracer scheduling
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit
        #self.solutions = {}


    def on_solution_callback(self):      
        self._solution_count += 1
        #Create dataframe to hold results
        results = []
        #results = {}
        #results = [{"Solution": self._solution_count}]
        #print('Solution %i' % self._solution_count)
        for d in self._num_days:
            #Get each day's schedule
            day = ["Day{}".format(d)]
            #day = {}
            #day = [{"Day":d}]
            #for d in range(self._num_days):
            #print('Day %i' % d)
            for n in self._num_nurses:
            #for n in range(self._num_nurses):
                is_working = False
                for s in self._num_shifts:
                #for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        #print('  Nurse %i works shift %i' % (n, s))
                        day.append([d,n,s,1])
                        #day.update({(d,n,s) : 1})
                        #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":1})
                if not is_working:
                    #print('  Nurse {} does not work'.format(n))
                    #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":0})
                    #day.update({(d,n,s) : 0})
                    day.append([d,n,s,0])
            #results.update({d : day})
            results.append(day)
        #self.solutions.update({self._solution_count : results})
        pd.DataFrame(results).to_csv("schedule_solution_{}.csv".format(self._solution_count),index=False)
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

In [32]:
#Save solutions as CSV
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
#Enumerate all solutions
solver.parameters.enumerate_all_solutions = True
#Display the first n solutions
solution_limit = 3
solution_printer = NursesPartialSolutionPrinter(x, tracers,
                                                days, sessions,
                                                solution_limit)
solver.Solve(model, solution_printer)
# Statistics.
print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())

Stop search after 3 solutions

Statistics
  - conflicts      : 0
  - branches       : 754
  - wall time      : 0.147643 s
  - solutions found: 3


In [33]:
### Create Gantt Graph of Solution ###

In [34]:
#Graph first solution
sch_data = pd.DataFrame([
    dict(Session=sess_times[0], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[1], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[2], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[3], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[4], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[5], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[6], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[7], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[8], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[9], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[10], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[11], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[12], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[13], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[14], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[15], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[16], Start=0, Finish=5, Tracer="NA", color="1")
])

#Graph solution
sch = pd.read_csv("schedule_solution_1.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
tracers_colors = ["2","3","4","5"]
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[3]) == 1:
                sch_entry = dict(Session=sess_times[int(ent_l[2])],
                                Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                Tracer=tracers_names[int(ent_l[1])], 
                                 color=tracers_colors[int(ent_l[1])])
                sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()

#fig.show()
file_name = 'scheduling_graph_sol_1'
fig.write_html(f"{file_name}.html")

In [35]:
#fig.to_dict() #use if want to adjust formatting

In [36]:
#Save Gantt Graph as HTML
file_name = 'scheduling_solution_1_graph'
fig.write_html(f"{file_name}.html")